## 1. 패키지 로드

In [1]:
import cv2
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from tqdm import tqdm_notebook
import re
import dlib

from img2vec_pytorch import Img2Vec
from PIL import Image
from sklearn.metrics.pairwise import cosine_similarity

import warnings # 경고문 무시
warnings.filterwarnings('ignore')

## 2. 클래스 정의

In [2]:
class CImg:
    def __init__(self):
        '''init class'''
        self.img2vec = Img2Vec(cuda=True)
  
    def img_to_rgb(self, image):
        '''convert image to rgb'''
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        return image

    def img_to_bgr(self, image):
        '''convert image to bgr'''
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        return image

    def img_adjust(self, image, brightness = 0, contrast = 30):
        '''adjust image'''
        image = np.int16(image)
        image = image * (contrast / 127 + 1) - contrast + brightness
        image = np.clip(image, 0, 255)
        image = np.uint8(image)

        return image

    def img_resize(self, image, size = 256):
        '''resize image'''
        image = cv2.resize(image, (size, size))

        return image
    
    def pil_to_cv(self, pil_image):
        '''PIL -> cv2'''
        numpy_image = np.array(pil_image)
        cv_image = cv2.cvtColor(numpy_image, cv2.COLOR_RGB2BGR)

        return cv_image
    
    def cv_to_pil(self, cv_image):
        '''cv2 -> PIL'''
        color_coverted = cv2.cvtColor(cv_image, cv2.COLOR_BGR2RGB)
        pil_image = Image.fromarray(color_coverted)
        return pil_image
    
    def img_face_align(self, image, size = 256, padding = 0.65):
        '''face align image'''
        detector = dlib.get_frontal_face_detector()
        sp = dlib.shape_predictor('../util/shape_predictor_5_face_landmarks.dat')
        dets = detector(image)
        if dets:
            pass
        else:
            #print("No detect face")
            return image
        s = sp(image, dets[0])
        image = dlib.get_face_chip(image, s, size=size, padding=padding)

        return image

    def img_compare(self, image_1, image_2):
        '''compare image'''
        image_1_vec = self.img2vec.get_vec(image_1, tensor=False)
        image_2_vec = self.img2vec.get_vec(image_2, tensor=False)
        
        image_1_vec = image_1_vec.reshape(1, -1)
        image_2_vec = image_2_vec.reshape(1, -1)
        
        return cosine_similarity(image_1_vec, image_2_vec)[0][0]


    def run_image_compare(self, image_1, image_2, size = 256, padding = 0.65, ratio = 0.75, brightness = 0, contrast = 30, show=False):
        '''compare image'''
        image_1 = self.img_to_rgb(image_1)
        image_1 = self.img_adjust(image_1, brightness=brightness, contrast=contrast)
        image_1 = self.img_resize(image_1, size=size)
        image_1 = self.img_face_align(image_1, size=size, padding=padding)
        image_1 = self.img_to_bgr(image_1)
        image_1 = self.cv_to_pil(image_1)
        
        image_2 = self.img_to_rgb(image_2)
        image_2 = self.img_adjust(image_2, brightness=brightness, contrast=contrast)
        image_2 = self.img_resize(image_2, size=size)
        image_2 = self.img_face_align(image_2, size=size, padding=padding)
        image_2 = self.img_to_bgr(image_2)
        image_2 = self.cv_to_pil(image_2)
        
        similarity = self.img_compare(image_1, image_2)

        return similarity

    def run(self, actor, webtoon, actor_image_path, webtoon_image_path):
        '''calculate all actor images and webtoon images similarity '''
        actor_df = actor.copy()
        webtoon_df = webtoon.copy()
        
        # webtoon name list and actor name list
        webtoon_names = webtoon_df['Title'].unique()
        actor_name_list = actor_df['Name'].to_list()
        actor_male_name_list = actor_df[actor_df['Sex'] == '남'].Name.to_list()
        actor_female_name_list = actor_df[actor_df['Sex'] == '여'].Name.to_list()
        
        # result dataframe
        result_df = actor_df[['Name', 'Age', 'Sex']]
        
        for webtoon_name in tqdm_notebook(webtoon_names):
            try:
                # get characters's names from webtoon name
                charater_names = webtoon_df[webtoon_df['Title'] == webtoon_name]['Character'].to_list()

                # calculate actors's images and characters's images
                for charater_name in charater_names:
                    try:
                        # character's image load 
                        character_image = cv2.imread(f'{webtoon_image_path}/{webtoon_name}_{charater_name}.jpg', flags=cv2.IMREAD_COLOR)

                        # get character's sex, age
                        character_sex = webtoon_df[webtoon_df['Character'] == charater_name]['Sex'].values[0]
                        character_age = webtoon_df[webtoon_df['Character'] == charater_name]['Age'].values[0]
                        character_age = int(re.sub('[^0-9]', '', character_age))
                    
                        # calculation according to sex
                        if character_sex == '남':
                            for actor_name in actor_male_name_list:
                                actor_age = int(actor_df[actor_df['Name'] == actor_name]['Age'].values[0])
                                if (character_age < actor_age) and (character_age + 20 > actor_age):
                                    actor_image = cv2.imread(f'{actor_image_path}/webtoon_{actor_name}.jpg', flags=cv2.IMREAD_COLOR)
                                    n_sim = self.run_image_compare(actor_image, character_image)
                                    result_df.loc[(result_df['Name'] == actor_name), f'{webtoon_name}_{charater_name}_image_score'] = n_sim
                        elif character_sex == '여':
                            for actor_name in actor_female_name_list:
                                actor_age = int(actor_df[actor_df['Name'] == actor_name]['Age'].values[0])
                                if (character_age < actor_age) and (character_age + 20 > actor_age):
                                    actor_image = cv2.imread(f'{actor_image_path}/webtoon_{actor_name}.jpg', flags=cv2.IMREAD_COLOR)
                                    n_sim = self.run_image_compare(actor_image, character_image)
                                    result_df.loc[(result_df['Name'] == actor_name), f'{webtoon_name}_{charater_name}_image_score'] = n_sim
                        else: # if, sex == NaN
                            for actor_name in actor_name_list:
                                actor_age = int(actor_df[actor_df['Name'] == actor_name]['Age'].values[0])
                                if (character_age < actor_age) and (character_age + 20 > actor_age):
                                    actor_image = cv2.imread(f'{actor_image_path}/webtoon_{actor_name}.jpg', flags=cv2.IMREAD_COLOR)
                                    n_sim = self.run_image_compare(actor_image, character_image)
                                    result_df.loc[(result_df['Name'] == actor_name), f'{webtoon_name}_{charater_name}_image_score'] = n_sim
                    except:
                        print(f'{charater_name} Error')
            except Exception as e:
                print(e)
                print(f'{webtoon_name} Error')
                
        
        return result_df

 ## 3. 이미지 로드 및 테스트

In [3]:
actor_image_path = '../image_data/Actor2Webtoon'
webtoon_image_path = '../image_data/Webtoon'

In [4]:
actor_image = cv2.imread(f'{actor_image_path}/webtoon_감우성.jpg')
webtoon_image = cv2.imread(f'{webtoon_image_path}/폭풍의전학생_주인공.jpg')

In [5]:
img = CImg()

In [6]:
img.run_image_compare(actor_image, webtoon_image)

0.66000986

## 4. for문을 통해 등장인물 - 배우 간 모든 이미지 유사도

In [7]:
actor_data = pd.read_csv('../text_data/Actor.csv', encoding='utf-8-sig')
webtoon_data = pd.read_csv('../text_data/Webtoon.csv', encoding='utf-8-sig')

In [8]:
result_df = img.run(actor_data, webtoon_data, actor_image_path, webtoon_image_path)

  0%|          | 0/71 [00:00<?, ?it/s]

공기영 Error
벽창오 Error
dj.8(팔Eight) Error
독고연 Error
최개발 Error
밥 Error
도동남 Error
우사미 Error
백지미 Error
붕어머리남자 Error
예민해 Error
점쟁이아가씨 Error
전진수 Error
Dr.에니그마 박사 Error
오강남 Error
종건 Error
동동배 Error
두태양 Error
이중현 Error
유수민 Error
주수애 Error
국사 선생님 Error
교장선생님 Error
김두만 Error
차택기 Error
사채업자 Error
상이 신세진 노인 Error
천존 Error
지동대신 Error
홍명희 Error
정소진 Error
이름없는링커 Error
라이트닝의 사이드킥 Error
지나 엘리슨(블러드팩) Error
한준성 Error
정지수 Error
우슬기 Error


libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile


국회의원 정국천 Error
최우남 Error
베네로 플란넬(하기스의 아버지) Error
김석훈 Error
패션왕 Error
시각 장애인 때밀이 Error
여탕 팀장 Error
여 회장 Error
꼬마 Error
하일권 Error
오지란(스피커) Error
한세진 Error
이수빈 Error
윤현석 Error
장호재 Error
송예린 Error
라자크 케르티아 Error
담임 선생님 Error
이백합 Error
장노란 Error
이태양 Error
김유리 Error
박현진 Error
송재민 Error
이경민 Error
윤지수 Error
이양선 Error
나유진 Error
박하늘 Error
이백합의 아버지 Error
이백합의 어머니 Error
진재현 Error
박예진 Error
김달수 Error
양채모 Error
마적두 Error
장영식 Error
박봉녀 Error
이물식 Error
박화수 Error
정명구 Error
한해나 Error
진서원 Error
이보겸 Error
윤채아 Error
김송이 Error
최율 Error
김지원 Error
현재희 Error
지아 Error
한유나 Error
한해나의 엄마 Error
한성호 Error
최율의 엄마 Error
최율의 아빠 Error
최율의 할아버지 Error
진서원의 엄마 Error
한유나의 전남친 Error
강은환 Error
소년 Error
막순이 Error
초영이 Error
조아연 Error
손우진 Error
장근재 Error
홍승복 Error
안동민 Error
김자홍 Error
진기한 Error
유성연 Error
중대장 Error
소대장 Error
염라대왕 Error
문오증 Error
육경열 Error
박수환 Error
이동은 Error
홍도연 Error
오동석 Error
김희진 Error
변동출 Error
조만섭 Error
힙제이 Error
김비서 Error
편희정 Error
최민규 Error
심봉희 Error
서민기 Error
권기혁 Error
이광재 Error
노준석 Error
권설하 Error
윤설하 Err

In [9]:
result_df.to_csv('../text_data/cos_sim_score_df.csv', encoding='utf-8-sig', index=False)

In [10]:
result_df.head()

,Name,Age,Sex,폭풍의전학생_주인공_image_score,폭풍의전학생_최완서_image_score,폭풍의전학생_김대용_image_score,폭풍의전학생_정해인_image_score,폭풍의전학생_황인규_image_score,폭풍의전학생_이연희_image_score,폭풍의전학생_조규식_image_score,...,노블레스_프랑켄슈타인_image_score,노블레스_레지스 K 란데그르_image_score,노블레스_세이라 J 로이아드_image_score,노블레스_라엘 케르티아_image_score,노블레스_카리어스 블러스터_image_score,노블레스_M-21_image_score,노블레스_타키오_image_score,노블레스_타오_image_score,전설의 레전드_전설_image_score,전설의 레전드_문태식_image_score
0,감우성,52,남,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.612988,NaN,NaN,NaN,NaN,NaN
1,강경준,39,남,NaN,NaN,NaN,NaN,NaN,NaN,0.586832,...,0.633482,NaN,NaN,0.533305,NaN,0.621947,0.544965,0.632591,NaN,0.725206
2,강기영,39,남,NaN,NaN,NaN,NaN,NaN,NaN,0.620132,...,0.658447,NaN,NaN,0.575278,NaN,0.642590,0.581335,0.662886,NaN,0.723405
3,강남길,64,남,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,강지섭,41,남,NaN,NaN,NaN,NaN,NaN,NaN,0.649044,...,0.689507,NaN,NaN,NaN,0.614106,NaN,NaN,NaN,NaN,NaN
